In [2]:
import pandas as pd


import pandas as pd
import glob

def process_dataset(file):
    d = pd.read_csv(file, sep=",")
    d.columns=['Name', 'Gender', 'Frequency']
    d['Year'] = int(file.split('/')[-1][3:7])
    return d

allfiles = glob.glob('Names/*.txt')
data = pd.concat(process_dataset(file) for file in allfiles)
data.head()


,Name,Gender,Frequency,Year
0,Helen,F,6811,1905
1,Margaret,F,5690,1905
2,Anna,F,5424,1905
3,Ruth,F,5068,1905
4,Elizabeth,F,4121,1905


In [3]:
def top_k_names_in_year(k, year):
    top_k = data.loc[data['Year']==year].nlargest(k, 'Frequency')
    result = {}
    for _, row in top_k.iterrows(): 
        result[row['Name']]= row['Frequency']
    return result

top_k_names_in_year(5, 1905)

{'Helen': 6811, 'James': 6042, 'John': 8060, 'Margaret': 5690, 'William': 6495}

In [4]:
def gender_name_frequencies(name):
    return {
        'M':data.loc[(data['Name']==name) & (data['Gender']=='M')].Frequency.sum(),
        'F':data.loc[(data['Name']==name) & (data['Gender']=='F')].Frequency.sum()
    }

gender_name_frequencies('John')

{'F': 21657, 'M': 5095674}

In [8]:
def top_k_names_in_year_relative_frequency(k, year):
    number_names = len(data.loc[data['Year']==year].Name.unique())
    top_k = data.loc[data['Year']==year].nlargest(k, 'Frequency')
    result = {}
    for _, row in top_k.iterrows(): 
        result[row['Name']]= row['Frequency']/number_names
    return result

top_k_names_in_year_relative_frequency(5, 1905)    

{'Helen': 2.044731311918343,
 'James': 1.8138697087961573,
 'John': 2.419693785649955,
 'Margaret': 1.708195737015911,
 'William': 1.9498649054338038}

In [89]:
# def gender_name_frequency_in_year(name, year):
#     m_cnt_vals = data.loc[(data['Name']==name) & (data['Gender']=='M') & (data['Year']==year), 'Frequency']
#     f_cnt_vals = data.loc[(data['Name']==name) & (data['Gender']=='F') & (data['Year']==year), 'Frequency']
#     m_cnt, f_cnt = 0,0
#     if m_cnt_vals.size>0:
#         m_cnt=m_cnt_vals[0]
#     if f_cnt_vals.size>0:
#         f_cnt=f_cnt_vals[0]  
#     return (m_cnt, f_cnt)   
data.loc[(data['Name']=='John') & (data['Gender']=='M') & (data['Year']==1905), 'Frequency']
    
#get all the multigender names    
    
data
a=0
for i in range(1858553):
    a+=1
print(a)
# # print(gender_name_frequency_in_year('Jamie', 1905))
# names = set()
print(len(data))

def gender_popularity_shifted_names():
    #make a df of name, year male gender frequency, female gender frequency, difference
    
    
    
    
#     a = 0
#     for _, _ in data.iterrows():
#         a+=1
#     print(a)
gender_popularity_shifted_names()



1858553


In [ ]:
count 
for index, row in data.iterrows():
    